In [10]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [11]:
from dotenv import load_dotenv

In [12]:
load_dotenv()

True

In [13]:
from openai import OpenAI

openai_client = OpenAI()

In [9]:
!uv add openai-agents==0.3.3

Resolved 164 packages in 790ms
Prepared 2 packages in 2.09s
Uninstalled 2 packages in 574ms
Installed 2 packages in 723ms
 - openai==2.2.0
 + openai==1.109.1
 - openai-agents==0.1.0
 + openai-agents==0.3.3


In [8]:
!uv remove openai

Resolved 164 packages in 490ms
Audited 144 packages in 150ms


In [1]:
import agents

In [2]:
agents.__version__

'0.3.3'

In [3]:
from agents import Agent, function_tool, Runner

In [5]:
import requests

In [42]:
def fetch_url(url: str) -> str:
    jina_reader_base_url = 'https://r.jina.ai/'
    jina_reader_url = jina_reader_base_url + url
    response = requests.get(jina_reader_url)
    return response.content.decode('utf-8')

In [43]:
import requests
from typing import Optional

reader_url_prefix = "https://r.jina.ai/"

def get_page_content(url: str) -> Optional[str]:
    """
    Fetch the Markdown content of a web page using the Jina Reader service.

    This function prepends the Jina Reader proxy URL to the provided `url`,
    sends a GET request with a timeout, and decodes the response as UTF-8 text.

    Args:
        url (str): The URL of the page to fetch.

    Returns:
        Optional[str]: The Markdown-formatted content of the page if the request
        succeeds; otherwise, None.

    Raises:
        None: All network or decoding errors are caught and suppressed.
               Logs or error messages could be added as needed.
    """
    reader_url = reader_url_prefix + url

    try:
        response = requests.get(reader_url, timeout=10)
        response.raise_for_status()  # raises for 4xx/5xx HTTP errors
        return response.content.decode("utf-8")
    except (requests.exceptions.RequestException, UnicodeDecodeError) as e:
        # Optional: log or print the error for debugging
        print(f"Error fetching content from {url}: {e}")
        return None


In [44]:
content = fetch_url('https://datatalks.club')

In [45]:
print(content)

Title: Welcome to DataTalks.Club

URL Source: https://datatalks.club/

Published Time: Thu, 16 Oct 2025 09:40:58 GMT

Markdown Content:
Welcome to DataTalks.Club


AI Dev Tools Zoomcamp: Learn AI-powered coding assistants and agents[Register here!](https://airtable.com/appJRFiWKHBgmEt70/shrpw7rk55Ewr1jCG)

DataTalks.Club
--------------

[Articles](https://datatalks.club/articles.html)[Slack](https://datatalks.club/slack.html)[Events](https://datatalks.club/events.html)[Podcast](https://datatalks.club/podcast.html)[Books](https://datatalks.club/books.html)[Courses](https://datatalks.club/blog/guide-to-free-online-courses-at-datatalks-club.html)

* * *

The place to talk about data

Global online community of data science professionals, ML engineers, and AI practitioners
-----------------------------------------------------------------------------------------

Subscribe to our weekly newsletter and join our Slack.

 We'll keep you informed about everything happening in the Club.

Email 


In [46]:
web_agent = Agent(
    name='web_agent',
    instructions="you're a helpful assistant",
    model="gpt-4o-mini",
    tools=[function_tool(fetch_url)]
    
)

In [47]:
from agents import Runner

In [48]:
runner = Runner()

In [49]:
question = "whats is this page about? https://openai.github.io/openai-agents-python/"

In [50]:
results = await runner.run(web_agent, input=question)

C:\Users\ryahj\AppData\Local\Temp\ipykernel_27776\3288517736.py:1: RuntimeWarning: coroutine 'Runner.run' was never awaited
  results = await runner.run(web_agent, input=question)


In [52]:
results.new_items

[ToolCallItem(agent=Agent(name='web_agent', handoff_description=None, tools=[FunctionTool(name='fetch_url', description='', params_json_schema={'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'title': 'fetch_url_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000024893DBDE40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)], mcp_servers=[], mcp_config={}, instructions="you're a helpful assistant", prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_headers=None

In [53]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)


In [54]:
await runner.run()

You: whats is this page about? https://openai.github.io/openai-agents-python/


You: what are agents?


You: stop


Chat ended.


In [55]:
## Youtube Video

In [56]:
from youtube_transcript_api import YouTubeTranscriptApi

def format_timestamp(seconds: float) -> str:
    """Convert seconds to H:MM:SS if > 1 hour, else M:SS"""
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours > 0:
        return f"{hours}:{minutes:02}:{secs:02}"
    else:
        return f"{minutes}:{secs:02}"


def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)


def fetch_transcript_raw(video_id):
    ytt_api = YouTubeTranscriptApi()
    transcript = ytt_api.fetch(video_id)
    return transcript


def fetch_transcript_text(video_id):
    transcript = fetch_transcript_raw(video_id)
    subtitles = make_subtitles(transcript)
    return subtitles 

In [61]:
from pathlib import Path

def fetch_transcript_cached(video_id):
    cache_dir = Path("../data_cache/youtube_videos")
    cache_dir.mkdir(parents=True, exist_ok=True)
    cache_file = cache_dir / f"{video_id}.txt"

    if cache_file.exists():
        return cache_file.read_text(encoding="utf-8")

    subtitles = fetch_transcript_text(video_id)
    cache_file.write_text(subtitles, encoding="utf-8")

    return subtitles


In [62]:
subtitles = fetch_transcript_cached('vK_SxyqIfwk')
print(subtitles[:500])

0:00 Hey everyone, welcome to our event. This
0:02 event is brought to you by data talks
0:03 club which is a community of people who
0:05 love data. We have weekly events today.
0:08 Uh this is one of such events. Um if you
0:11 want to find out more about the events
0:13 we have, there is a link in the
0:14 description. Um so click on that link,
0:16 check it out right now. We actually have
0:19 quite a few events in our pipeline, but
0:21 we need to put them on the website. Uh
0:24 but keep a


In [63]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return fetch_transcript_cached(video_id)

In [64]:
summary_instructions = """
You're helpful assistant that helps answer user qusetions
about Youtube videos
"""

In [65]:
youtube_agent = Agent(
    name='youtube_agent',
    instructions=summary_instructions,
    tools=[function_tool(fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

In [66]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=web_agent
)

In [ ]:
await runner.run();

You: Whats this video about https://www.youtube.com/watch?v=vK_SxyqIfwk&t=172s
